## IRS Innovative Assignment

Roll no.: 21bce046 Vartika Desai and 21bce234 Pranshu Desai

Spotify Music Recommender System

In [ ]:
import requests
import base64

# Your Spotify API credentials
client_id = '5159a1a08319416389c59d7a1910310f'
client_secret = 'c4aa8b99aab04612bf92a54057f2a355'

# Function to get an access token
def get_access_token(client_id, client_secret):
    # Spotify URL for authentication
    auth_url = 'https://accounts.spotify.com/api/token'

    # Encode as Base64
    message = f"{client_id}:{client_secret}"
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii')

    headers = {
        'Authorization': f'Basic {base64_message}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body = {
        'grant_type': 'client_credentials',
    }

    # POST request to get the access token
    response = requests.post(auth_url, headers=headers, data=body)
    token = response.json().get('access_token')
    return token

# Function to get playlist details by its ID
def get_playlist_by_id(token, playlist_id):
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}'
    headers = {
        'Authorization': f'Bearer {token}',
    }

    # GET request to fetch the playlist
    response = requests.get(playlist_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

This one is cold start

In [1]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# # Initialize Spotipy with your credentials
# client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # Function to fetch a list of genres from Spotify
# def get_genres():
#     genres = sp.recommendation_genre_seeds()
#     return genres['genres']

# # Function to fetch a list of artists in a given genre
# def get_artists_in_genre(genre):
#     artists = []
#     results = sp.search(q=f'genre:"{genre}"', type='artist', limit=50)
#     for artist in results['artists']['items']:
#         artists.append(artist['name'])
#     return artists

# # Function to allow the user to choose genres and artists
# def choose_genres_and_artists():
#     print("Welcome to the Spotify Song Recommender!")
#     print("Choose three genres from the following list:")
#     genres = get_genres()
#     for i, genre in enumerate(genres):
#         print(f"{i+1}. {genre}")

#     selected_genres = []
#     while len(selected_genres) < 3:
#         try:
#             choice = int(input("Enter the number of the genre: "))
#             if 1 <= choice <= len(genres):
#                 selected_genres.append(genres[choice - 1])
#             else:
#                 print("Invalid choice. Please choose a number between 1 and", len(genres))
#         except ValueError:
#             print("Invalid input. Please enter a number.")

#     print("\nGreat! Now choose three artists from each selected genre:")
#     selected_artists = []
#     for genre in selected_genres:
#         print(f"\nArtists in {genre}:")
#         artists = get_artists_in_genre(genre)
#         for i, artist in enumerate(artists):
#             print(f"{i+1}. {artist}")

#         while len(selected_artists) < 3 * len(selected_genres):
#             try:
#                 choice = int(input(f"Enter the number of an artist in {genre}: "))
#                 if 1 <= choice <= len(artists):
#                     selected_artists.append(artists[choice - 1])
#                     break
#                 else:
#                     print("Invalid choice. Please choose a number between 1 and", len(artists))
#             except ValueError:
#                 print("Invalid input. Please enter a number.")

#     return selected_genres, selected_artists

# # Example usage
# selected_genres, selected_artists = choose_genres_and_artists()
# print("\nSelected genres:", selected_genres)
# print("Selected artists:", selected_artists)


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Get comma separated list of genres from selected_genres
selected_genres = ",".join(selected_genres)

# Get artist name from selected_artists
artist_name = selected_artists[0]  # Assuming you only want recommendations based on the first artist

# Search for artist using their name
search_results = sp.search(q=artist_name, type="artist")
artist_id = search_results["artists"]["items"][0]["id"]

# Use the artist's ID and genres to get recommendations
recommendations = sp.recommendations(seed_artists=[artist_id], seed_genres=[selected_genres])

# Process the recommendations data (e.g., track names, artists)
for track in recommendations["tracks"]:
    track_name = track["name"]
    artist = track["artists"][0]["name"]
    # Add your logic to display or use the recommendations here
    print(f"Recommended Track: {track_name} by {artist}")


Recommended Track: Silent Way by Milo Greene
Recommended Track: Hey There Delilah by Plain White T's
Recommended Track: Never Say Never by Justin Bieber
Recommended Track: Drunk by Ed Sheeran
Recommended Track: Poison & Wine by The Civil Wars
Recommended Track: Sexual Healing by Ben Harper
Recommended Track: Bluebird by Jamestown Revival
Recommended Track: Murder in the City by Brandi Carlile
Recommended Track: True North by Caroline Spence
Recommended Track: Everything You Want by Boyce Avenue
Recommended Track: Starships - BBC Live Version by Conor Maynard
Recommended Track: July (feat. Leon Bridges) by Noah Cyrus
Recommended Track: Carry Me by The Secret Sisters
Recommended Track: Hear the Noise That Moves So Soft and Low by James Vincent McMorrow
Recommended Track: Mean (feat. Megan Nicole) by Boyce Avenue
Recommended Track: Dirt Around the Tree by Candi Carpenter
Recommended Track: Ho Hey by Nashville Cast
Recommended Track: In the Stream by S. Carey
Recommended Track: February Se

getting playlist_id from link

In [ ]:
import re

def extract_playlist_id(link):
    # Regular expression pattern to match the playlist ID
    pattern = r'playlist/(\w+)\?'

    # Using re.search to find the pattern in the link
    match = re.search(pattern, link)

    if match:
        # Extracting the playlist ID group
        playlist_id = match.group(1)
        return playlist_id
    else:
        return None

# Function to check if the user has a playlist or they are new
def check_user_status():
    while True:
        choice = input("Do you have a playlist you want to use for recommendations? (yes/no): ").lower()
        if choice == 'yes':
            spotify_link = input("Enter the link to your playlist: ")
            playlist_id = extract_playlist_id(spotify_link)
            if playlist_id:
                return playlist_id
            else:
                print("Failed to extract playlist ID from the provided link. Please try again.")
        elif choice == 'no':
            print("Okay, let's proceed with the cold start procedure.")
            return None
        else:
            print("Invalid choice. Please enter 'yes' or 'no'.")

# Check user status
playlist_id = check_user_status()

# Main flow
if playlist_id:
    access_token = get_access_token(client_id, client_secret)
    playlist_details = get_playlist_by_id(access_token, playlist_id)

    if playlist_details:
        print(f"Found playlist: {playlist_details['name']}")
        # You can add more details to print here, based on what information you need
    else:
        print("Playlist not found or failed to fetch playlist details.")
else:
    # Proceed with cold start procedure
    selected_genres, selected_artists = choose_genres_and_artists()
    print("\nSelected genres:", selected_genres)
    print("Selected artists:", selected_artists)


Found playlist: Darci Mix


In [ ]:
import requests

def get_playlist_tracks(token, playlist_id):
    tracks_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(tracks_url, headers=headers)
    track_items = response.json().get('items', [])
    track_ids = [item['track']['id'] for item in track_items]
    return track_ids

def get_tracks_features(token, track_ids):
    features_url = "https://api.spotify.com/v1/audio-features"
    headers = {'Authorization': f'Bearer {token}'}
    features = []

    for track_id in track_ids:
        response = requests.get(f"{features_url}/{track_id}", headers=headers)
        if response.status_code == 200:
            features.append(response.json())

    return features


playlist_id = '37i9dQZF1EIYoJVDSpf3PB'
track_ids = get_playlist_tracks(access_token, playlist_id)
track_features = get_tracks_features(access_token, track_ids)

print("Track Features:", track_features)


Track Features: [{'danceability': 0.767, 'energy': 0.593, 'key': 4, 'loudness': -5.901, 'mode': 1, 'speechiness': 0.0413, 'acousticness': 0.0324, 'instrumentalness': 4.5e-06, 'liveness': 0.0878, 'valence': 0.579, 'tempo': 130.007, 'type': 'audio_features', 'id': '0F4FejjWi3bf5vGOkuQwPS', 'uri': 'spotify:track:0F4FejjWi3bf5vGOkuQwPS', 'track_href': 'https://api.spotify.com/v1/tracks/0F4FejjWi3bf5vGOkuQwPS', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0F4FejjWi3bf5vGOkuQwPS', 'duration_ms': 171692, 'time_signature': 4}, {'danceability': 0.645, 'energy': 0.534, 'key': 6, 'loudness': -10.8, 'mode': 0, 'speechiness': 0.0479, 'acousticness': 0.157, 'instrumentalness': 0, 'liveness': 0.0863, 'valence': 0.463, 'tempo': 105.02, 'type': 'audio_features', 'id': '3azJifCSqg9fRij2yKIbWz', 'uri': 'spotify:track:3azJifCSqg9fRij2yKIbWz', 'track_href': 'https://api.spotify.com/v1/tracks/3azJifCSqg9fRij2yKIbWz', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3azJifCSqg9fRij2yK

In [ ]:
import pandas as pd

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(track_features)
df = df.drop(columns=['id', 'uri', 'track_href', 'analysis_url', 'type'])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Define the variational autoencoder model
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2_mean = nn.Linear(256, latent_dim)
        self.fc2_logvar = nn.Linear(256, latent_dim)
        self.fc3 = nn.Linear(latent_dim, 256)
        self.fc4 = nn.Linear(256, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mean = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        recon = torch.sigmoid(self.fc4(h))
        return recon

    def forward(self, x):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        recon = self.decode(z)
        return recon, mean, logvar

# Custom dataset class
class SongDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

# Function to preprocess and scale the data
def preprocess_data(df):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler

# Hyperparameters
input_dim = len(df.columns)  # Dimensionality of input data
latent_dim = 16  # Dimensionality of latent space
batch_size = 64
epochs = 50
learning_rate = 1e-3

# Preprocess data
data, scaler = preprocess_data(df)

# Split data into train and validation sets
train_loader = DataLoader(SongDataset(data), batch_size=batch_size, shuffle=True)

# Instantiate the VAE model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(input_dim, latent_dim).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        recon_batch, mean, logvar = model(batch)

        # Compute the reconstruction loss and KL divergence
        recon_loss = criterion(recon_batch, batch)
        kl_div = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())

        # Total loss
        loss = recon_loss + kl_div

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss / len(train_loader.dataset)}')

# Save the model if needed
# torch.save(model.state_dict(), 'vae_model.pth')


Epoch [1/50], Loss: 0.6784235382080078
Epoch [2/50], Loss: 0.5000907897949218
Epoch [3/50], Loss: 0.3922697067260742
Epoch [4/50], Loss: 0.32932796478271487
Epoch [5/50], Loss: 0.28860925674438476
Epoch [6/50], Loss: 0.25538225173950196
Epoch [7/50], Loss: 0.2238941192626953
Epoch [8/50], Loss: 0.194005069732666
Epoch [9/50], Loss: 0.16662517547607422
Epoch [10/50], Loss: 0.14384602546691894
Epoch [11/50], Loss: 0.1271159267425537
Epoch [12/50], Loss: 0.11567452430725098
Epoch [13/50], Loss: 0.10910266876220703
Epoch [14/50], Loss: 0.10452279090881347
Epoch [15/50], Loss: 0.1004721450805664
Epoch [16/50], Loss: 0.09560024261474609
Epoch [17/50], Loss: 0.0902723503112793
Epoch [18/50], Loss: 0.08432295799255371
Epoch [19/50], Loss: 0.0789878273010254
Epoch [20/50], Loss: 0.07477984428405762
Epoch [21/50], Loss: 0.07109780311584472
Epoch [22/50], Loss: 0.06830432891845703
Epoch [23/50], Loss: 0.0660777187347412
Epoch [24/50], Loss: 0.06399056911468506
Epoch [25/50], Loss: 0.0615618133544

In [ ]:
import numpy as np

# Function to generate samples within the latent space
def generate_latent_samples(model, num_samples):
    with torch.no_grad():
        z_samples = torch.randn(num_samples, latent_dim).to(device)
        generated_samples = model.decode(z_samples)
    return generated_samples.cpu().numpy()

# Function to decode latent samples and inverse transform scaled data
def decode_samples(samples, scaler):
    decoded_samples = scaler.inverse_transform(samples)
    return decoded_samples

# Generate samples in the latent space
num_samples = 1000
latent_samples = generate_latent_samples(model, num_samples)

# Decode latent samples to obtain feature values
decoded_samples = decode_samples(latent_samples, scaler)

# Calculate the range of favorable features
feature_ranges = {
    feature: (np.min(decoded_samples[:, i]), np.max(decoded_samples[:, i]))
    for i, feature in enumerate(df.columns)
}

print("Range of favorable features:")
for feature, (min_val, max_val) in feature_ranges.items():
    print(f"{feature}: {min_val:.2f} to {max_val:.2f}")


Range of favorable features:
danceability: 0.62 to 0.66
energy: 0.63 to 0.65
key: 5.74 to 6.57
loudness: -6.83 to -6.43
mode: 0.49 to 0.60
speechiness: 0.06 to 0.08
acousticness: 0.15 to 0.19
instrumentalness: 0.01 to 0.02
liveness: 0.17 to 0.19
valence: 0.33 to 0.36
tempo: 120.62 to 126.18
duration_ms: 185337.55 to 195553.97
time_signature: 4.00 to 4.05


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Initialize Spotipy with your credentials
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define a function to search for tracks based on features
def search_tracks(features):
    query = " ".join([f"{feature}:{value}" for feature, value in features.items()])
    results = sp.search(q=query, type='track', limit=10)  # You can adjust the limit as needed
    return results['tracks']['items']

# Define the favorable features obtained from your decoded samples
favorable_features = {
    'danceability': (0.62, 0.66),
    'energy': (0.63, 0.65),
    'key': (5.74, 6.57),
    'loudness': (-6.83, -6.43),
    'mode': (0.49, 0.60),
    'speechiness': (0.06, 0.08),
    'acousticness': (0.15, 0.19),
    'instrumentalness': (0.01, 0.02),
    'liveness': (0.17, 0.19),
    'valence': (0.33, 0.36),
    'tempo': (120.62, 126.18),
    'duration_ms': (185337.55, 195553.97),
    'time_signature': (4.00, 4.05)  # Special case for time_signature
}

# Search for tracks based on favorable features
similar_tracks = search_tracks(favorable_features)

# Print out the names and artists of the similar tracks
print("Similar tracks:")
for track in similar_tracks:
    print(f"{track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")
    print(f"Spotify URI: {track['uri']}")
    print()


Similar tracks:


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import concurrent.futures

# Set up your Spotify credentials
client_id = 'your_client_id'
client_secret = 'your_client_secret'

# Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to get tracks from a playlist
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Function to recommend similar tracks based on a seed track
def recommend_similar_tracks(seed_track_id):
    track_info = sp.track(seed_track_id)
    artist_id = track_info['artists'][0]['id']
    similar_tracks = sp.artist_related_artists(artist_id)
    recommended_tracks = []
    for artist in similar_tracks['artists']:
        top_tracks = sp.artist_top_tracks(artist['id'])
        recommended_tracks.extend(top_tracks['tracks'])
    return recommended_tracks[:10]  # Limit to 10 recommended tracks

# Main function to recommend similar songs from a playlist
def recommend_similar_songs_from_playlist(playlist_id):
    playlist_tracks = get_playlist_tracks(playlist_id)
    seed_track_ids = [track['track']['id'] for track in playlist_tracks if track['track']]
    recommended_songs = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Get similar songs concurrently
        futures = [executor.submit(recommend_similar_tracks, track_id) for track_id in seed_track_ids]
        for future in concurrent.futures.as_completed(futures):
            recommended_songs.extend(future.result())
            if len(recommended_songs) >= 10:  # Limit to 10 recommended songs
                break

    return recommended_songs[:10]  # Return at most 10 recommended songs

# Example usage
playlist_id = '37i9dQZF1EIYoJVDSpf3PB'
recommended_songs = recommend_similar_songs_from_playlist(playlist_id)
for song in recommended_songs:
    print(song['name'], '-', song['artists'][0]['name'])


Next - Shaker
THE DEVIL WEARS PRADA - PLVTINUM
In the Dark - Shaker
ALLADAT - PLVTINUM
Nowhere - Shaker
Pretty Lies - Shaker
Give It Up - Shaker
XXX - Shaker
In The Middle - Shaker
PLAYED TO LOSE - Shaker


In [ ]:
# Function to get tracks from a playlist
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Function to recommend similar tracks based on a seed track
def recommend_similar_tracks(seed_track_id):
    track_info = sp.track(seed_track_id)
    artist_id = track_info['artists'][0]['id']
    similar_tracks = sp.artist_related_artists(artist_id)
    recommended_tracks = []
    for artist in similar_tracks['artists']:
        top_tracks = sp.artist_top_tracks(artist['id'])
        recommended_tracks.extend(top_tracks['tracks'])
    return recommended_tracks

# Main function to recommend similar songs from a playlist
def recommend_similar_songs_from_playlist(playlist_id):
    playlist_tracks = get_playlist_tracks(playlist_id)
    seed_track_ids = [track['track']['id'] for track in playlist_tracks if track['track']]
    similar_songs = []
    for track_id in seed_track_ids:
        similar_songs.extend(recommend_similar_tracks(track_id))
    return similar_songs

# Example usage
playlist_id = '37i9dQZF1EIYoJVDSpf3PB'
recommended_songs = recommend_similar_songs_from_playlist(playlist_id)
for song in recommended_songs:
    print(song['name'], '-', song['artists'][0]['name'])

Next - Shaker
THE DEVIL WEARS PRADA - PLVTINUM
In the Dark - Shaker
ALLADAT - PLVTINUM
Nowhere - Shaker
Pretty Lies - Shaker
Give It Up - Shaker
XXX - Shaker
In The Middle - Shaker
PLAYED TO LOSE - Shaker
BADASF - Sani Knight
Paranoid - Sani Knight
Dark Place - Sani Knight
Heaven Sent - Sani Knight
Ride for You - Sani Knight
WHERE'D U GO - Sani Knight
Sensitive - Sani Knight
Pink Floyd - Sani Knight
Leave Behind - Bosquet
Nocturnal - Sani Knight
Hopeless - Always Never
Wylin' - Always Never
Patterns - Always Never
Millions - Always Never
Can't Imagine - Always Never
Bad for Me - Always Never
It's Over - Always Never
Don't Say It - Always Never
Canadian Dubai - Always Never
Tragedy - Always Never
Eyes On You - SWIM
Body Loud - SWIM
Lose Myself - SWIM
Like A Drug - Jordyne
Liability - SWIM
Thin Lines - SWIM
Snow White - sobhhï
Trust - SWIM
Alive - SWIM
Promises - SWIM
LoveYou - Dxvn.
Drowning - Dxvn.
Weekend - Dxvn.
Nightfall - Dxvn.
love is evil - Dxvn.
Tonight - Dxvn.
Catch Me - Dxvn.
